# 基于VGG-16实现中草药分类

In [1]:
import os
import zipfile
import random
import json
import paddle
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from paddle.io import Dataset

In [2]:

'''
参数配置
'''
train_parameters = {
    "input_size": [3, 224, 224],  #输入图片的shape
    "class_dim": -1,  #分类数
    "src_path": "E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\data\\Chinese Medicine.zip",  #原始数据集路径
    "target_path": "E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\data\\dataset",  #要解压的路径
    "train_list_path": "E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\data\\train.txt",  #train.txt路径
    "eval_list_path": "E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\data\\eval.txt",  #eval.txt路径
    "readme_path": "E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\data\\readme.json",  #readme.json路径
    "label_dict": {},  #标签字典
    "num_epochs": 1,  #训练轮数
    "train_batch_size": 8,  #训练时每个批次的大小
    "skip_steps": 10,
    "save_steps": 30,
    "learning_strategy": {  #优化函数相关的配置
        "lr": 0.0001  #超参数学习率
    },
    "checkpoints": "E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\checkpoints"  #保存的路径

}

## 数据准备

In [3]:
def unzip_data(src_path, target_path):
    '''
    解压原始数据集，将src_path路径下的zip包解压至target_path目录下
    '''
    if (not os.path.isdir(target_path + "Chinese Medicine")):
        z = zipfile.ZipFile(src_path, 'r')
        z.extractall(path=target_path)
        z.close()

In [15]:

def get_data_list(target_path, train_list_path, eval_list_path):
    '''
    生成数据列表
    '''
    #存放所有类别的信息
    class_detail = []
    #获取所有类别保存的文件夹名称
    data_list_path = target_path + "\\Chinese Medicine\\"
    class_dirs = os.listdir(data_list_path)
    #总的图像数量
    all_class_images = 0
    #存放类别标签
    class_label = 0
    #存放类别数目
    class_dim = 0
    #存储要写进eval.txt和train.txt中的内容
    trainer_list = []
    eval_list = []
    #读取每个类别，['river', 'lawn','church','ice','desert']
    for class_dir in class_dirs:
        if class_dir != ".DS_Store":
            class_dim += 1
            #每个类别的信息
            class_detail_list = {}
            eval_sum = 0
            trainer_sum = 0
            #统计每个类别有多少张图片
            class_sum = 0
            #获取类别路径
            path = data_list_path + class_dir
            # 获取所有图片
            img_paths = os.listdir(path)
            for img_path in img_paths:  # 遍历文件夹下的每个图片
                name_path = path + '/' + img_path  # 每张图片的路径
                if class_sum % 8 == 0:  # 每8张图片取一个做验证数据
                    eval_sum += 1  # test_sum为测试数据的数目
                    eval_list.append(name_path + "\t%d" % class_label + "\n")
                else:
                    trainer_sum += 1
                    trainer_list.append(name_path + "\t%d" % class_label + "\n")  #trainer_sum测试数据的数目
                class_sum += 1  #每类图片的数目
                all_class_images += 1  #所有类图片的数目

            # 说明的json文件的class_detail数据
            class_detail_list['class_name'] = class_dir  #类别名称
            class_detail_list['class_label'] = class_label  #类别标签
            class_detail_list['class_eval_images'] = eval_sum  #该类数据的测试集数目
            class_detail_list['class_trainer_images'] = trainer_sum  #该类数据的训练集数目
            class_detail.append(class_detail_list)
            #初始化标签列表
            train_parameters['label_dict'][str(class_label)] = class_dir
            class_label += 1

    #初始化分类数
    train_parameters['class_dim'] = class_dim

    #乱序
    random.shuffle(eval_list)
    with open(eval_list_path, 'a') as f:
        for eval_image in eval_list:
            f.write(eval_image)

    random.shuffle(trainer_list)
    with open(train_list_path, 'a') as f2:
        for train_image in trainer_list:
            f2.write(train_image)

    # 说明的json文件信息
    readjson = {}
    readjson['all_class_name'] = data_list_path  #文件父目录
    readjson['all_class_images'] = all_class_images
    readjson['class_detail'] = class_detail
    jsons = json.dumps(readjson, sort_keys=True, indent=4, separators=(',', ': '))
    with open(train_parameters['readme_path'], 'w') as f:
        f.write(jsons)
    print('生成数据列表完成！')

In [16]:
'''
参数初始化
'''
src_path = train_parameters['src_path']
target_path = train_parameters['target_path']
train_list_path = train_parameters['train_list_path']
eval_list_path = train_parameters['eval_list_path']

'''
解压原始数据到指定路径
'''
unzip_data(src_path, target_path)

'''
划分训练集与验证集，乱序，生成数据列表
'''
#每次生成数据列表前，首先清空train.txt和eval.txt
with open(train_list_path, 'w') as f:
    f.seek(0)
    f.truncate()
with open(eval_list_path, 'w') as f:
    f.seek(0)
    f.truncate()

#生成数据列表
get_data_list(target_path, train_list_path, eval_list_path)

生成数据列表完成！


In [17]:
class dataset(Dataset):
    def __init__(self, data_path, mode='train'):
        """
        数据读取器
        :param data_path: 数据集所在路径
        :param mode: train or eval
        """
        super().__init__()
        self.data_path = data_path
        self.img_paths = []
        self.labels = []

        if mode == 'train':
            with open(os.path.join(self.data_path, "train.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
            for img_info in self.info:
                img_path, label = img_info.strip().split('\t')
                self.img_paths.append(img_path)
                self.labels.append(int(label))

        else:
            with open(os.path.join(self.data_path, "eval.txt"), "r", encoding="utf-8") as f:
                self.info = f.readlines()
            for img_info in self.info:
                img_path, label = img_info.strip().split('\t')
                self.img_paths.append(img_path)
                self.labels.append(int(label))

    def __getitem__(self, index):
        """
        获取一组数据
        :param index: 文件索引号
        :return:
        """
        # 第一步打开图像文件并获取label值
        img_path = self.img_paths[index]
        img = Image.open(img_path)
        if img.mode != 'RGB':
            img = img.convert('RGB')
        img = img.resize((224, 224), Image.BILINEAR)
        img = np.array(img).astype('float32')
        img = img.transpose((2, 0, 1)) / 255
        label = self.labels[index]
        label = np.array([label], dtype="int64")
        return img, label

    def print_sample(self, index: int = 0):
        print("文件名", self.img_paths[index], "\t标签值", self.labels[index])

    def __len__(self):
        return len(self.img_paths)

In [20]:
#训练数据加载
train_dataset = dataset('E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\data', mode='train')
train_loader = paddle.io.DataLoader(train_dataset, batch_size=16, shuffle=True)
#测试数据加载
eval_dataset = dataset('E:\\Python\\BaiDuPaddle\\基于VGG-16实现中草药分类\\data', mode='eval')
eval_loader = paddle.io.DataLoader(eval_dataset, batch_size=8, shuffle=False)

## 2.模型搭建

In [25]:
class ConvPool(paddle.nn.Layer):
    def __init__(self, num_channels, num_filters, filter_size, pool_size, pool_stride, groups, conv_stride=1,
                 conv_padding=1):
        super(ConvPool, self).__init__()

        for i in range(groups):
            self.add_sublayer(
                "conv_%d" % i,
                paddle.nn.Conv2D(in_channels=num_channels, out_channels=num_filters, kernel_size=filter_size,
                                 stride=conv_stride, padding=conv_padding)
            )
            self.add_sublayer(
                "relu_%d" % i,
                paddle.nn.ReLU()
            )
            num_channels = num_filters
        self.add_sublayer(
            "max_pool",
            paddle.nn.MaxPool2D(
                kernel_size=pool_size,
                stride=pool_stride

            )
        )

    def forward(self, inputs):
        x = input
        for prefix, sub_layer in self.named_children():
            x = sub_layer(x)
        return x

In [26]:
class VGGNet(paddle.nn.Layer):
    def __init__(self):
        super(VGGNet, self).__init__()
        self.conv_pool1 = ConvPool(3, 64, 3, 2, 2, 2)
        self.conv_pool2 = ConvPool(64, 128, 3, 2, 2, 2)
        self.conv_pool3 = ConvPool(128, 256, 3, 2, 2, 3)
        self.conv_pool4 = ConvPool(256, 512, 3, 2, 2, 3)
        self.conv_pool5 = ConvPool(256, 512, 3, 2, 2, 3)
        self.fc1 = paddle.nn.Linear(7 * 7 * 512, 4096)
        self.fc2 = paddle.nn.Linear(4096, 4096)
        self.fc3 = paddle.nn.Linear(4096, train_parameters['class_dim'])

    def forward(self, input, label=None):
        x = self.convpool01(input)
        x = self.convpool02(x)
        x = self.convpool03(x)
        x = self.convpool04(x)
        x = self.convpool05(x)

        x = paddle.reshape(x, shape=[-1, 512 * 7 * 7])
        x = self.fc01(x)
        x = self.fc02(x)
        x = self.fc03(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label=label)
            return x, acc
        else:
            return x

In [27]:
def draw_process(title, color, iters, data, label):
    plt.title(title, fontsize=24)
    plt.xlabel("iter", fontsize=20)
    plt.ylabel(label, fontsize=20)
    plt.plot(iters, data, color=color, label=label)
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
model = VGGNet()
model.train()
cross_entropy = paddle.nn.CrossEntropyLoss()
optimizer = paddle.optimizer.Adam(learning_rate=train_parameters['learning_strategy']['lr'],
                                  parameters=model.parameters())

steps = 0
Iters, total_loss, total_acc = [], [], []

for epo in range(train_parameters['num_epochs']):
    for _, data in enumerate(train_loader()):
        steps += 1
        x_data = data[0]
        y_data = data[1]
        predicts, acc = model(x_data, y_data)
        loss = cross_entropy(predicts, y_data)
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        if steps % train_parameters["skip_steps"] == 0:
            Iters.append(steps)
            total_loss.append(loss.numpy()[0])
            total_acc.append(acc.numpy()[0])
            #打印中间过程
            print('epo: {}, step: {}, loss is: {}, acc is: {}' \
                  .format(epo, steps, loss.numpy(), acc.numpy()))
        #保存模型参数
        if steps % train_parameters["save_steps"] == 0:
            save_path = train_parameters["checkpoints"] + "/" + "save_dir_" + str(steps) + '.pdparams'
            print('save model to: ' + save_path)
            paddle.save(model.state_dict(), save_path)
paddle.save(model.state_dict(), train_parameters["checkpoints"] + "/" + "save_dir_final.pdparams")
draw_process("trainning loss", "red", Iters, total_loss, "trainning loss")
draw_process("trainning acc", "green", Iters, total_acc, "trainning acc")